In [1]:
import os
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("CoffeeData") \
    .master("local[*]") \
    .getOrCreate()

spark


In [3]:
import pandas as pd

df_pd = pd.read_csv(
    r"C:\Users\HP\Desktop\Coffee\data\cleaned\coffee-cleaned.csv",
    sep=';',             
    encoding='utf-8',    
)
print(df_pd.shape)
df_pd.head()

(2215, 14)


,coffee_name,total_score,roaster_location,coffee_origin,roast_level,est._price,agtron_ground,agtron_roast,price_per_100g,price_per_ounce,origin_country,clean_roaster_location,roaster_country,origin_country_clean
0,Colombia Finca Campo Hermosa,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$29.99/8 ounces,62.0,82.0,13.22,29.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
1,Colombia Finca La Sirena Mango Co-Ferment,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$22.99/8 ounces,63.0,87.0,10.14,22.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
2,In Bloom,94.0,"Jersey City, New Jersey","Colombia, Ethiopia",Light,$25.00/250 grams,64.0,88.0,10.00,0.881834,Ethiopia,"Jersey, New Jersey",USA,Ethiopia
3,Ethiopia Washed Kaffa Gimbo Lot Rich Espresso,96.0,"Chia-Yi, Taiwan","Gimbo, Kaffa Province, Ethiopia",Medium Light,NT $250/8 ounces,55.0,77.0,110.23,250.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia
4,Ethiopia Natural Gute Bona,95.0,"Chia-Yi, Taiwan","Sidamo growing region, southern Ethiopia",Medium Light,NT $400/8 ounces,62.0,78.0,176.37,400.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia


In [14]:
df_pyspark = spark.read.csv('coffee-cleaned.csv', sep = ';')

In [5]:
type(df_pyspark)

pyspark.sql.classic.dataframe.DataFrame

In [15]:
df_pyspark.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)



In [18]:
df_pyspark = df_pyspark.na.drop() 
df_pyspark.show() 

+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|                 _c0|        _c1|                 _c2|                 _c3|         _c4|              _c5|          _c6|         _c7|           _c8|            _c9|          _c10|                _c11|           _c12|                _c13|
+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|         coffee_name|total_score|    roaster_location|       coffee_origin| roast_level|       est._price|agtron_ground|agtron_roast|price_per_100g|price_per_ounce|origin_country|clean_roaster_loc...|roaster_country|origin_country_clean|
|Colombia Finca Ca...|         94|Cleveland,

In [19]:
df_pyspark = df_pyspark.drop('coffee_origin', 'est._price', 'price_per_ounce', 'roaster_location', 'origin_country')
df_pyspark.show()

+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|                 _c0|        _c1|                 _c2|                 _c3|         _c4|              _c5|          _c6|         _c7|           _c8|            _c9|          _c10|                _c11|           _c12|                _c13|
+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|         coffee_name|total_score|    roaster_location|       coffee_origin| roast_level|       est._price|agtron_ground|agtron_roast|price_per_100g|price_per_ounce|origin_country|clean_roaster_loc...|roaster_country|origin_country_clean|
|Colombia Finca Ca...|         94|Cleveland,

In [21]:
df_pyspark.drop('coffee_origin', 'est._price', 'price_per_ounce', 'roaster_location', 'origin_country').show()

+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|                 _c0|        _c1|                 _c2|                 _c3|         _c4|              _c5|          _c6|         _c7|           _c8|            _c9|          _c10|                _c11|           _c12|                _c13|
+--------------------+-----------+--------------------+--------------------+------------+-----------------+-------------+------------+--------------+---------------+--------------+--------------------+---------------+--------------------+
|         coffee_name|total_score|    roaster_location|       coffee_origin| roast_level|       est._price|agtron_ground|agtron_roast|price_per_100g|price_per_ounce|origin_country|clean_roaster_loc...|roaster_country|origin_country_clean|
|Colombia Finca Ca...|         94|Cleveland,

In [ ]:
вирішити питання шапки та назви колонок